# Music/Speech Classifier

In [ ]:
import attk
import os
import csv
import numpy as np
import librosa
import timeit
import random
import subprocess
import urllib2
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
from IPython.display import display, Audio
import pandas as pd

!mkdir -p /sharedfolder/music_classifier/

os.chdir('/sharedfolder/music_classifier/')

training_audio_dir_name = "Music"

class_dir_pathname = '/sharedfolder/music_classifier/_classes_' + training_audio_dir_name

subprocess.call(['mkdir', '-p', class_dir_pathname])

In [ ]:
## Downloading male and female training data
# (Uncomment the commands below first.)

os.chdir('/sharedfolder/music_classifier/')

!wget -N http://xtra.arloproject.com/datasets/audio/Mozilla_CV_UBM_Subset/females_5k.zip
!wget -N http://xtra.arloproject.com/datasets/audio/Mozilla_CV_UBM_Subset/males_5k.zip
!wget -N http://xtra.arloproject.com/datasets/aapb-ubm/Male_AAPB_171110.zip
!wget -N http://xtra.arloproject.com/datasets/aapb-ubm/Female_AAPB_171110.zip

!unzip -o females_5k.zip
!unzip -o males_5k.zip
!unzip -o Male_AAPB_171110.zip
!unzip -o Female_AAPB_171110.zip

In [ ]:
## Downloading music training data
# (Uncomment the commands below first.)

os.chdir('/sharedfolder/music_classifier/')

!wget -N http://www.stephenmclaughlin.net/hipstas/audio/FMA_train_clips_set_1.zip
!wget -N http://www.stephenmclaughlin.net/hipstas/audio/Music_Larry_Monroe.zip

!unzip -o FMA_train_clips_set_1.zip
!unzip -o Music_Larry_Monroe.zip

In [ ]:
## Creating directories for our two classes

music_path = os.path.join(class_dir_pathname, 'Music')
non_music_path = os.path.join(class_dir_pathname, 'Non-Music')

subprocess.call(['mkdir', music_path])
subprocess.call(['mkdir', non_music_path])

In [ ]:
## Consolidating all non-music clips in a single directory

!mv females_5k/* '/sharedfolder/music_classifier/_classes_Music/Non-Music'
!mv males_5k/* '/sharedfolder/music_classifier/_classes_Music/Non-Music'
!mv Male_AAPB_171110/* '/sharedfolder/music_classifier/_classes_Music/Non-Music'
!mv Female_AAPB_171110/* '/sharedfolder/music_classifier/_classes_Music/Non-Music'

In [ ]:
## Consolidating all music clips in a single directory

!mv FMA_train_clips_set_1/* '/sharedfolder/music_classifier/_classes_Music/Music'
!mv Music_Larry_Monroe/* '/sharedfolder/music_classifier/_classes_Music/Music'

In [ ]:
%%capture
## Extracting features (MFCCs, deltas, and delta-deltas) from all clips, then writing features to CSVs
# (Note that we are not extracting vowel segments like we do with speech classifiers.)

os.chdir(class_dir_pathname)

for dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
    print("> Starting " + dir_name)
    try:
        os.chdir(os.path.join(class_dir_pathname, dir_name))
        try: os.mkdir('./_mfccs_and_deltas')
        except: pass
        filenames = [item for item in os.listdir('./') if item[-4:].lower()=='.wav']
        for filename in filenames:
            csv_out_path = './_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv'
            if not os.path.isfile(csv_out_path):
                try:
                    mfccs = attk.get_mfccs_and_deltas(filename, n_mfcc=30, n_fft=4096, freq_min=100, freq_max=16000)
                    if len(mfccs) > 0:
                        with open(csv_out_path, 'w') as fo:
                            csv_writer = csv.writer(fo)
                            csv_writer.writerows(mfccs)  
                except Exception as e:
                    print('FILE ERROR: ' + filename)
                    print(e)
    except Exception as e:
        print('SKIPPING DIRECTORY: ' + dir_name)     ## Skipping class directories for which we didn't extract vowels
        #print(e)

In [ ]:
## Continue to the next notebook to train and run the music classifier.